# Projeto 1 - Objetos

Disciplina SCC0250 - Computação Gráfica

<hr>

João Pedro Ribeiro da Silva - 12563727

Código baseado naquele desenvolvido e disponibilizado pelo professor na Aula 09 - Prática sobre Model e View 

## Bibliotecas

In [2624]:
import glfw
from OpenGL.GL import *
import numpy as np
import glm
import math
from numpy import random

from shader_s import Shader

In [2625]:
# from PIL import Image

## Inicializando janela

In [2626]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE)

altura = 700
largura = 700

window = glfw.create_window(largura, altura, "Programa", None, None)

if (window == None):
    print("Failed to create GLFW window")
    glfwTerminate()
    
glfw.make_context_current(window)

## Constroi e compila os shaders. Também "linka" eles ao programa

#### Novidade aqui: modularização dessa parte do código --- temos agora uma classe e arquivos próprios para os shaders (vs e fs)
Créditos: https://learnopengl.com

In [2627]:
ourShader = Shader("vertex_shader.vs", "fragment_shader.fs")
ourShader.use()

program = ourShader.getProgram()

## Preparando dados para enviar a GPU

Até aqui, compilamos nossos Shaders para que a GPU possa processá-los.

Por outro lado, as informações de vértices geralmente estão na CPU e devem ser transmitidas para a GPU.


### Carregando Modelos (vértices e texturas) a partir de Arquivos

A função abaixo carrega modelos a partir de arquivos no formato WaveFront (.obj).

Para saber mais sobre o modelo, acesse: https://en.wikipedia.org/wiki/Wavefront_.obj_file

In [2628]:
global vertices_list
vertices_list = []    
global textures_coord_list
textures_coord_list = []


def load_model_from_file(filename):
    """Loads a Wavefront OBJ file. """
    objects = {}
    vertices = []
    texture_coords = []
    faces = []

    material = None

    # abre o arquivo obj para leitura
    for line in open(filename, "r"): ## para cada linha do arquivo .obj
        if line.startswith('#'): continue ## ignora comentarios
        values = line.split() # quebra a linha por espaço
        if not values: continue

        ### recuperando vertices
        if values[0] == 'v':
            vertices.append(values[1:4])

        ### recuperando coordenadas de textura
        elif values[0] == 'vt':
            texture_coords.append(values[1:3])

        ### recuperando faces 
        elif values[0] in ('usemtl', 'usemat'):
            material = values[1]
        elif values[0] == 'f':
            face = []
            face_texture = []
            for v in values[1:]:
                w = v.split('/')
                face.append(int(w[0]))
                if len(w) >= 2 and len(w[1]) > 0:
                    face_texture.append(int(w[1]))
                else:
                    face_texture.append(0)

            faces.append((face, face_texture, material))

    model = {}
    model['vertices'] = vertices
    model['texture'] = texture_coords
    model['faces'] = faces

    return model


def load_texture_from_file(texture_id, img_textura):
    glBindTexture(GL_TEXTURE_2D, texture_id)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    img = Image.open(img_textura)
    img_width = img.size[0]
    img_height = img.size[1]
    image_data = img.tobytes("raw", "RGB", 0, -1)
    #image_data = np.array(list(img.getdata()), np.uint8)
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, img_width, img_height, 0, GL_RGB, GL_UNSIGNED_BYTE, image_data)



'''
É possível encontrar, na Internet, modelos .obj cujas faces não sejam triângulos. Nesses casos, precisamos gerar triângulos a partir dos vértices da face.
A função abaixo retorna a sequência de vértices que permite isso. Créditos: Hélio Nogueira Cardoso e Danielle Modesti (SCC0650 - 2024/2).
'''
def circular_sliding_window_of_three(arr):
    if len(arr) == 3:
        return arr
    circular_arr = arr + [arr[0]]
    result = []
    for i in range(len(circular_arr) - 2):
        result.extend(circular_arr[i:i+3])
    return result


def load_obj_and_texture(objFile, texturesList=[]):
    modelo = load_model_from_file(objFile)
    
    ### inserindo vertices do modelo no vetor de vertices
    verticeInicial = len(vertices_list)
    print('Processando modelo {}. Vertice inicial: {}'.format(objFile, len(vertices_list)))
    faces_visited = []
    for face in modelo['faces']:
        if face[2] not in faces_visited:
            faces_visited.append(face[2])
        for vertice_id in circular_sliding_window_of_three(face[0]):
            vertices_list.append(modelo['vertices'][vertice_id - 1])
        for texture_id in circular_sliding_window_of_three(face[1]):
            textures_coord_list.append(modelo['texture'][texture_id - 1])
        
    verticeFinal = len(vertices_list)
    print('Processando modelo {}. Vertice final: {}'.format(objFile, len(vertices_list)))
    
    ### carregando textura equivalente e definindo um id (buffer): use um id por textura!
    for id in range(len(texturesList)):
        load_texture_from_file(id,texturesList[id])
    
    return verticeInicial, verticeFinal - verticeInicial

### Vamos carregar cada modelo e definir funções para desenhá-los

In [2629]:
# carrega objetos
vertice_inicial_fantasma, num_vertices_fantasma = load_obj_and_texture('objetos/fantasma.obj') # não há texturas para esse objeto (carrega apenas .obj)
vertice_inicial_arvore, num_vertices_arvore = load_obj_and_texture('objetos/arvore.obj') # não há texturas para esse objeto (carrega apenas .obj)
vertice_inicial_grama, num_vertices_grama = load_obj_and_texture('objetos/grama.obj') # não há texturas para esse objeto (carrega apenas .obj)
vertice_inicial_ceu, num_vertices_ceu = load_obj_and_texture('objetos/ceu.obj') # não há texturas para esse objeto (carrega apenas .obj)
vertice_inicial_lua, num_vertices_lua = load_obj_and_texture('objetos/lua.obj') # não há texturas para esse objeto (carrega apenas .obj)
vertice_inicial_cabana, num_vertices_cabana = load_obj_and_texture('objetos/cabana.obj') # não há texturas para esse objeto (carrega apenas .obj)
vertice_inicial_caderno, num_vertices_caderno = load_obj_and_texture('objetos/caderno.obj') # não há texturas para esse objeto (carrega apenas .obj)

def model_objeto(vertice_inicial, num_vertices, t_x=0, t_y=0, t_z=0, s_x=1, s_y=1, s_z=1, r_x=0, r_y=0, r_z=0):
    # aplica a matriz model
    
    mat_model = model(t_x, t_y, t_z,  # translação
                      s_x, s_y, s_z,  # escala
                      r_x, r_y, r_z)  # rotação
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)

def desenha_objeto(vertice_inicial, num_vertices, cor=[1,1,1,1]):
    # cor
    loc_color = glGetUniformLocation(program, "color")
    glUniform4f(loc_color, cor[0], cor[1], cor[2], cor[3])
       
    # desenha o objeto
    glDrawArrays(GL_TRIANGLES, vertice_inicial, num_vertices) ## renderizando

Processando modelo objetos/fantasma.obj. Vertice inicial: 0
Processando modelo objetos/fantasma.obj. Vertice final: 1338
Processando modelo objetos/arvore.obj. Vertice inicial: 1338
Processando modelo objetos/arvore.obj. Vertice final: 1518
Processando modelo objetos/grama.obj. Vertice inicial: 1518
Processando modelo objetos/grama.obj. Vertice final: 1524
Processando modelo objetos/ceu.obj. Vertice inicial: 1524
Processando modelo objetos/ceu.obj. Vertice final: 1530
Processando modelo objetos/lua.obj. Vertice inicial: 1530
Processando modelo objetos/lua.obj. Vertice final: 1566
Processando modelo objetos/cabana.obj. Vertice inicial: 1566
Processando modelo objetos/cabana.obj. Vertice final: 1581
Processando modelo objetos/caderno.obj. Vertice inicial: 1581
Processando modelo objetos/caderno.obj. Vertice final: 1641


### Para enviar nossos dados da CPU para a GPU, precisamos requisitar dois slots (buffers): um para os vértices e outro para as texturas.

In [2630]:
buffer_VBO = glGenBuffers(1)

### Enviando coordenadas de vértices para a GPU

Veja os parâmetros da função glBufferData [https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glBufferData.xhtml]

In [2631]:
vertices = np.zeros(len(vertices_list), [("position", np.float32, 3)])
vertices['position'] = vertices_list


# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer_VBO)
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_STATIC_DRAW)
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)
loc_vertices = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc_vertices)
glVertexAttribPointer(loc_vertices, 3, GL_FLOAT, False, stride, offset)

## Câmera fixa

Matrizes View e Projection são constantes

In [2632]:
# view
cameraPos   = glm.vec3(0.0, 0.0, -10.0)
cameraFront = glm.vec3(0.0, 0.0, 1.0)
cameraUp    = glm.vec3(0.0, 1.0, 0.0)

MAT_VIEW = np.array(
    glm.lookAt(cameraPos, cameraPos + cameraFront, cameraUp)
)
loc_view = glGetUniformLocation(program, "view")
glUniformMatrix4fv(loc_view, 1, GL_TRUE, MAT_VIEW)

# projection
fov   =  45.0

MAT_PROJECTION = np.array(
    glm.perspective(glm.radians(fov), largura/altura, 0.1, 100.0)
)
loc_projection = glGetUniformLocation(program, "projection")
glUniformMatrix4fv(loc_projection, 1, GL_TRUE, MAT_PROJECTION)   

## Matriz Model

In [2633]:
def model(t_x=0, t_y=0, t_z=0, s_x=1, s_y=1, s_z=1, r_x=0, r_y=0, r_z=0):
    
    matrix_transform = glm.mat4(1.0) # instanciando uma matriz identidade
       
    # aplicando translacao (terceira operação a ser executada)
    matrix_transform = glm.translate(matrix_transform, glm.vec3(t_x, t_y, t_z))
    
    # aplicando rotacao (segunda operação a ser executada)
    # eixo x
    matrix_transform = glm.rotate(matrix_transform, math.radians(r_x), glm.vec3(1, 0, 0))
    
    # eixo y
    matrix_transform = glm.rotate(matrix_transform, math.radians(r_y), glm.vec3(0, 1, 0))
    
    # eixo z
    matrix_transform = glm.rotate(matrix_transform, math.radians(r_z), glm.vec3(0, 0, 1))
    
    # aplicando escala (primeira operação a ser executada)
    matrix_transform = glm.scale(matrix_transform, glm.vec3(s_x, s_y, s_z))
    
    matrix_transform = np.array(matrix_transform)
    
    return matrix_transform

### Eventos de Teclado

In [2634]:
rotacao_fantasma = 169 - 180
escala_arvore = 2.5
caderno_mexendo = False
caderno_amostra = False 
caderno_velocidade = 0
caderno_altura = 0

def key_event(window,key,scancode,action,mods):
    global rotacao_fantasma, escala_arvore, caderno_mexendo, caderno_velocidade
    
    if key == glfw.KEY_RIGHT: 
        rotacao_fantasma -= 2.0
            
    elif key == glfw.KEY_LEFT: 
        rotacao_fantasma += 2.0
        
    if key == glfw.KEY_UP:
        escala_arvore += 0.1
        if escala_arvore > 3.0:  # limite opcional
            escala_arvore = 3.0

    elif key == glfw.KEY_DOWN:
        escala_arvore -= 0.1
        if escala_arvore < 0.1:  # limite mínimo
            escala_arvore = 0.1
    
    if key == 32 and action == glfw.PRESS and not caderno_mexendo:
        if caderno_amostra:
            caderno_velocidade = -0.1
            caderno_mexendo = True
        else:
            caderno_velocidade = 0.1
            caderno_mexendo = True

glfw.set_key_callback(window,key_event)


## Nesse momento, nós exibimos a janela!


In [2635]:
glfw.show_window(window)

## Loop principal da janela.

In [ ]:
glEnable(GL_DEPTH_TEST) ### importante para 3D


def multiplica_matriz(a,b):
    m_a = a.reshape(4,4)
    m_b = b.reshape(4,4)
    m_c = np.dot(m_a,m_b)
    c = m_c.reshape(1,16)
    return c
   
while not glfw.window_should_close(window):
    glfw.poll_events() 
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    glClearColor(1.0, 1.0, 1.0, 1.0)

    # glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)

    #objetos 2D

    #GRAMA
    model_objeto(vertice_inicial_grama, num_vertices_grama)
    desenha_objeto(vertice_inicial_grama, num_vertices_grama, cor=[0.231,0.329,0.066,1])

    #CEU
    model_objeto(vertice_inicial_ceu, num_vertices_ceu)
    desenha_objeto(vertice_inicial_ceu, num_vertices_ceu, cor=[0,0,0.27,1])

    #LUA
    model_objeto(vertice_inicial_lua, num_vertices_lua, t_x=-3, t_y=4, t_z=4)
    desenha_objeto(vertice_inicial_lua, num_vertices_lua, cor=[0.764,0.764,0.764,1])

    #CABANA
    model_objeto(vertice_inicial_cabana, num_vertices_cabana, t_x=0, t_y=1, t_z=4)
    #parede
    desenha_objeto(vertice_inicial_cabana, 6, cor=[0.411,0.247,0.145,1])
    #porta
    desenha_objeto(vertice_inicial_cabana + 6, 6, cor=[0.6,0.317,0.188,1])
    #teto
    desenha_objeto(vertice_inicial_cabana + 12, num_vertices_cabana-12, cor=[0.219, 0.145, 0.12,1])

    #CADERNO
    model_objeto(vertice_inicial_caderno, num_vertices_caderno, t_x=0, t_y=-7+caderno_altura, t_z=0, s_x=-3, s_y=3, s_z=3)
    desenha_objeto(vertice_inicial_caderno, 6, cor=[1,1,1,1])
    desenha_objeto(vertice_inicial_caderno+6, num_vertices_caderno-6, cor=[0,0,0,1])
    
    #objetos 3D
    
    # desenhando fantasma
    model_objeto(vertice_inicial_fantasma, num_vertices_fantasma, t_x=-2, t_z=2, r_y=rotacao_fantasma)
    # corpo
    desenha_objeto(vertice_inicial_fantasma, 1320, cor=[0.470,0.427,0.545,1])
    # olho
    desenha_objeto(vertice_inicial_fantasma + 1320, 6,  cor=[1,1,1,1])
    #boca
    desenha_objeto(vertice_inicial_fantasma + 1326, 12,  cor=[0.760,0.529,0.647,1])

    # desenhando arvore
    model_objeto(vertice_inicial_arvore, num_vertices_arvore, t_x=2, t_y=-1.5, t_z=2, s_x=escala_arvore, s_y=escala_arvore, s_z=escala_arvore)
    ## tronco
    desenha_objeto(vertice_inicial_arvore, 121, cor=[0.219, 0.145, 0.12, 0.35])
    ## copa
    desenha_objeto(vertice_inicial_arvore+120, num_vertices_arvore-120, cor=[0.043, 0.211, 0.09, 0.4])
    
    #movimento caderno
    print(caderno_altura)
    if caderno_mexendo:
        caderno_altura += caderno_velocidade
        if caderno_altura > 5:
            caderno_altura = 5
            caderno_mexendo = False
            caderno_amostra = True
        if caderno_altura < 0:
            caderno_altura = 0
            caderno_mexendo = False
            caderno_amostra = False
    
    glfw.swap_buffers(window)

glfw.terminate()

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
